# Data Preprocessing

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
dataset=pd.read_csv('flipkart_data.csv', encoding='unicode_escape')
dataset.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [4]:
data=dataset[["product_name","product_category_tree"]]
print(data.shape)
data.head()

(20000, 2)


,product_name,product_category_tree
0,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl..."
1,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B..."
2,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >..."
3,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl..."
4,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care..."


In [5]:
data.isnull().sum()

product_name             0
product_category_tree    0
dtype: int64

In [6]:
data=data.drop_duplicates()
data.shape

(12916, 2)

### Extract product category from breadcrumbs

In [7]:
rows=[]

for i in data.index:
    tree=data['product_category_tree'][i]
    ltIndex=tree.find('>>')
    if ltIndex == -1:
        continue
    category=tree[2:ltIndex]
    rows.append({"product_name":data['product_name'][i],"category_name":category})
data=pd.DataFrame(rows)
data.head()

,category_name,product_name
0,Clothing,Alisha Solid Women's Cycling Shorts
1,Furniture,FabHomeDecor Fabric Double Sofa Bed
2,Footwear,AW Bellies
3,Pet Supplies,Sicons All Purpose Arnica Dog Shampoo
4,Footwear,"dilli bazaaar Bellies, Corporate Casuals, Casuals"


In [8]:
print("Number of classes of categories:",data['category_name'].nunique())

Number of classes of categories: 32


### NLTK for NLP

In [9]:
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
nltk.download('punkt') # tokenizer
nltk.download('stopwords') # 'useless words'
nltk.download('wordnet') # a library for finding synonyms antonyms and so on
nltk.download('averaged_perceptron_tagger') # tagging data
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sanjisingh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sanjisingh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sanjisingh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sanjisingh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\sanjisingh\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [10]:
import string

def filterPunctuations(tokens):
    '''Removes punctuations from the array of tokens and returns the resultant array'''
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    return words

In [11]:
def processProductName(product):

    # break the sentence into words
    tokens = word_tokenize(product)
    #print(tokens)
    
    #remove punctuation
    tokens = filterPunctuations(tokens)
    #print(tokens)

    
    #convert to lower case
    tokens = [w.lower() for w in tokens]
    #print(tokens)

    #remove stopwords
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    words=list(filter(lambda word:word not in stop_words,tokens))   

    
    #lemmatize the words
    wordnet_lemmatizer=WordNetLemmatizer()
    words=[wordnet_lemmatizer.lemmatize(word) for word in words]
    #print(words)
    
    #unique words
    words=pd.Series(words).drop_duplicates().tolist()
    return words

In [12]:
rows=[]

for index, row in data.iterrows(): 
    product=row["product_name"]
    words=processProductName(product)
    rows.append({"product_name":product,"token_list":words,"token":' '.join(words),"category_name":row["category_name"]})
df=pd.DataFrame(rows)
df.head()

,category_name,product_name,token,token_list
0,Clothing,Alisha Solid Women's Cycling Shorts,alisha solid woman cycling short,"[alisha, solid, woman, cycling, short]"
1,Furniture,FabHomeDecor Fabric Double Sofa Bed,fabhomedecor fabric double sofa bed,"[fabhomedecor, fabric, double, sofa, bed]"
2,Footwear,AW Bellies,aw belly,"[aw, belly]"
3,Pet Supplies,Sicons All Purpose Arnica Dog Shampoo,sicons purpose arnica dog shampoo,"[sicons, purpose, arnica, dog, shampoo]"
4,Footwear,"dilli bazaaar Bellies, Corporate Casuals, Casuals",dilli bazaaar belly corporate casuals,"[dilli, bazaaar, belly, corporate, casuals]"


# Training Test Split

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X=df.iloc[:,2]
y=df.iloc[:,0]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
vectorizer = CountVectorizer()
X_train_cv=vectorizer.fit_transform(X_train)
transformer = TfidfTransformer()
X_train_tfidf = transformer.fit_transform(X_train_cv)
X_test_cv=vectorizer.transform(X_test)
X_test_tfidf = transformer.transform(X_test_cv)

In [17]:
def vectorize(value):
    cv=vectorizer.transform(value)
    tfidf = transformer.transform(cv)
    return tfidf

## Classifier Selection

In [18]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


models={"Naive Bayes":MultinomialNB(),"SVM":SVC(C=1.0, kernel='linear', gamma='auto'),"KNN":KNeighborsClassifier(),"Decision Tree":DecisionTreeClassifier()}

In [19]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.metrics import f1_score

cv = StratifiedKFold(n_splits=10)
scores=[]
for model in models:
    clf=models[model]
    clf.fit(X_train_tfidf, y_train)
    y_pred=clf.predict(X_test_tfidf)
    #print(f1_score(y_test.values,y_pred,average="micro"))
    f1=cross_val_score(clf, X_train_tfidf, y_train, cv=cv,scoring="f1_micro")
    scores.append({"classifier":model,"cross-validated F1 score":f1.mean()})
scores_df=pd.DataFrame(scores)
scores_df.head()

,classifier,cross-validated F1 score
0,Naive Bayes,0.857349
1,SVM,0.946325
2,KNN,0.920409
3,Decision Tree,0.920904


In [22]:
clf=models["SVM"]

In [23]:
y_pred=clf.predict(X_test_tfidf)
print(f1_score(y_test.values,y_pred,average="micro"))

0.9500358594310304


## Prediction

In [24]:
def predict_category(productName):
    X_test=pd.DataFrame([productName],columns=["product_name"])
    results=[]
    for index, row in X_test.iterrows(): 
        product=row["product_name"]
        words=' '.join(processProductName(product))
        results.append(words)
    X_test=vectorize(results)
    return clf.predict(X_test)[0]

In [59]:
category=predict_category(input("Enter the product name : "))
print("The predicted category is: ",category)

Enter the product name : 38024PP25 Minimalists Analog Watch - For Men & Women
The predicted category is:  Watches 


## Predicting e-commerce data

#### Transaction Data

In [40]:
transaction_df=pd.read_csv('transactions.csv', encoding='unicode_escape')
transaction_df.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,0,536365,85123A,JJC LH-62 Lens Hood,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,1,536365,71053,Jerry's Jppb1151b M Pet Bed,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,2,536365,84406B,Printland CMW1628 Ceramic Mug,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,3,536365,84029G,Smart Picks Pouch Cartoons Art Polyester Penci...,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,4,536365,84029E,Super Drool Plains Plant Container Set,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [41]:
# preprocessing
transaction_df=transaction_df.dropna()
transaction_df=transaction_df.drop_duplicates()
transaction_df=transaction_df[transaction_df["Quantity"]>0]

#### Merchants Data

In [28]:
merchants=pd.read_csv('merchants.csv')
merchants.head()

,Merchant_Name,Category,Ad
0,Generic Clothing,Clothing,A collection of the best clothing we have to o...
1,Generic Jewellery,Jewellery,A collection of the best jewellery we have to ...
2,Generic Automotive,Automotive,A collection of the best automotive we have to...
3,Generic Footwear,Footwear,A collection of the best footwear we have to o...
4,Generic Home Decor & Festive Needs,Home Decor & Festive Needs,A collection of the best home decor & festive ...


### Recommender System

In [35]:
customers_df=transaction_df.CustomerID.value_counts().rename_axis('customer_id').reset_index(name='counts')
customers_df.head(n=10)

,customer_id,counts
0,17841.0,7847
1,14911.0,5677
2,14096.0,5111
3,12748.0,4596
4,14606.0,2700
5,15311.0,2379
6,14646.0,2080
7,13089.0,1818
8,13263.0,1677
9,14298.0,1637


In [30]:
def find_top_categories(df,customer_id,n):
    customer=df[df["CustomerID"]==customer_id]
    customer=customer[["Description","Quantity","CustomerID"]]
    rows=[]
    for index, row in customer.iterrows(): 
        product=row["Description"]
        rows.append({"product_name":product,"category":predict_category(product),"quantity":row["Quantity"]})
    customer_df=pd.DataFrame(rows)
    top_categories=customer_df.category.value_counts().rename_axis('categories').reset_index(name='counts').categories[:n]
    return list(top_categories)

In [31]:
def categorize(customers_df,num_customers,num_categories):
    top_customers=[int(c) for c in customers_df.customer_id[:num_customers]]
    rows=[]
    for cid in top_customers:
        rows.append({"customer_id":cid, "top_categories":find_top_categories(transaction_df,cid,num_categories)})
    return pd.DataFrame(rows)

In [33]:
top_list=categorize(customers_df,10,5)

In [34]:
top_list

,customer_id,top_categories
0,17841,"[Beauty and Personal Care , Kitchen & Dining ,..."
1,14911,"[Beauty and Personal Care , Kitchen & Dining ,..."
2,14096,"[Beauty and Personal Care , Kitchen & Dining ,..."
3,12748,"[Beauty and Personal Care , Kitchen & Dining ,..."
4,14606,"[Beauty and Personal Care , Kitchen & Dining ,..."
5,15311,"[Beauty and Personal Care , Toys & School Supp..."
6,14646,"[Beauty and Personal Care , Bags, Wallets & Be..."
7,13089,"[Beauty and Personal Care , Bags, Wallets & Be..."
8,13263,"[Beauty and Personal Care , Kitchen & Dining ,..."
9,14298,"[Beauty and Personal Care , Home Decor & Festi..."
